In [ ]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import sys
import re
import logging
import os
from pathlib import Path
from functools import partial
from itertools import chain
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots

import bhutils
from bhutils import bhutilspy as bh
# from api.ezutils import bh, mkt_defs, ccy_cals, mx_cals, mtg_dts, gens

from gioutils.utils import parse_offset, today, get_next_n, get_bday
from gioutils.gui.base import View, ViewStore, WidgetView, Tabs
from gioutils.blpw import BlpQuery
from gioutils.utils import RepeatedTimer
from gioutils import ezutils as ez
from gioutils.ezutils import bh, bhs, SQL, ZCILSHelper, get_price_info, get_id, LiveInflLocal, COBInflLocal
from gioutils.gui.base import ViewStore
from gioutils.blpw import BlpParser
from gioutils.gui.datagrid import DataGridGio

from api.data.base import data_path, radar_path
from api.gui.params import bbg_params, spot_params, fwd_params
from api.gui.theme import bg_color, grid_style
from api.collectors import CollectorStoreMixin, SpotForward, Futures, Market, Mtgs, FwdMatrix, VolMatrix, ECOPred, ZCILMatrix, ECO
from api.gui.editable_grid import EditableGrid
from api.gui.price_grids import PricingGrid
from api.gui.grids import SpotGrid, FutureGrid, MtgGrid, MatGrid, GroupMktGrid # GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
from api.gui.global_config import GlobalConfig
from api.gui.monitors import MktMonitor, VolMonitor, ECOMonitor
from api.app import DataCollector
from api.gui.renderers import RendererStoreMixin, RendererComponentStoreMixin


pd.options.plotting.backend = "plotly"

In [4]:
import numpy as np
import pandas as pd
import ipywidgets as w
import ipyvuetify as v
from ipydatagrid import DataGrid

def make_df(rows=20, cols=2):
    df = pd.DataFrame(np.random.uniform(size=(rows, cols))).round(2)
    return df

n_grids = 1
n_tabs = 2

grids = [[DataGridGio(make_df()) for i in range(n_grids)] for j in range(n_tabs)]

widgets = [
    [
        v.Btn(children=['Test with a different widget'], class_="ml-2")
        for i in range(n_grids)
    ] for j in range(n_tabs)
]

children = [
    w.VBox(
        [
            w.HBox(grids[i]),
            w.HBox(widgets[i]),
        ] 
    ) for i in range(n_tabs)
]
tab = w.Tab()
tab.children = children
for i in range(n_tabs):
    tab.set_title(i, f'tab{i}')

tabs_bar = v.Tabs(
    # dark=True,
    v_model=None,
    children=[v.Tab(children=[f'tab{i}']) for i in range(n_tabs)],
)

tabs_items = v.TabsItems(
    v_model=None,
    # dark=True,
    children=[
        v.TabItem(
            children=[
                w.VBox(
                    [
                        w.HBox(grids[i]),
                        w.HBox(widgets[i]),
                    ] 
                ) 
            ]
        ) for i in range(n_tabs)
    ],
)

v_tab = v.Container(
    class_="my-0 py-0",
    fluid=True,
    children=[
        v.Row(children=[v.Col(children=[tabs_bar])]),
        v.Row(children=[v.Col(children=[tabs_items])]),
    ]
)
w.jslink((tabs_bar, 'v_model'), (tabs_items, 'v_model'))

btn = v.Btn(children=['click to change layout'], outlined=True, color='primary')
def on_click_change_layout(widget, event, payload):
    grids[1][0].layout = {
        'width': '300px', 
        'height': '300px', 
        # 'align-content': 'center'
    }
    widgets[1][0].layout = {
        'width': '500px', 
        # 'height': '300px', 
        # 'align-content': 'center'
    }
btn.on_event('click', on_click_change_layout)

w.VBox(
    [
        btn,
        v.Row(
            children=[
                v.Col(children=[v.Html(children=['ipywidgets'], tag='h1'), tab]), 
                v.Col(children=[v.Html(children=['ipyvuetify'], tag='h1'), v_tab])
            ]
        )
    ]
)

In [6]:
grids[1][0].rerender()

In [ ]:
grids[1][0].data = grids[1][0].data

In [ ]:
grids[1][0].